In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=False)

Mounted at /content/gdrive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dropout, Conv1D, GlobalMaxPooling1D, Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model

In [3]:
# 규원님이 전처리 완료하신 최종 데이터 (+라벨까지 붙은)
df = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/deeplearning_NLP/perfume/word embedding_hyun/data/dataset_210626_215600.csv')
df.drop('Unnamed: 0', axis=1, inplace=True)
len(df)

74779

In [4]:
df.head()

,gender,name,accords,review,tokenized,only_english,longer_than_2_A,stopwords_removed,lemmatizated,label
0,female,Alien Mugler for women,"['white floral', 'amber', 'woody']","Got a sample of this today, and my 9 year old ...","['got', 'a', 'sample', 'of', 'this', 'today', ...","['got', 'a', 'sample', 'of', 'this', 'today', ...","['got', 'sample', 'this', 'today', 'and', 'yea...","['got', 'sample', 'today', 'year', 'old', 'dau...","['get', 'sample', 'today', 'year', 'old', 'dau...",2.0
1,female,Alien Mugler for women,"['white floral', 'amber', 'woody']",First impressions. Test on blotter.\r\nI have ...,"['first', 'impressions.', 'test', 'on', 'blott...","['first', 'impressions', 'test', 'on', 'blotte...","['first', 'impressions', 'test', 'blotter', 'h...","['first', 'impressions', 'test', 'blotter', 'h...","['first', 'impression', 'test', 'blotter', 'he...",2.0
2,female,Alien Mugler for women,"['white floral', 'amber', 'woody']",This perfume reminds me of my best friend. Act...,"['this', 'perfume', 'reminds', 'me', 'of', 'my...","['this', 'perfume', 'reminds', 'me', 'of', 'my...","['this', 'perfume', 'reminds', 'best', 'friend...","['perfume', 'reminds', 'best', 'friend', 'actu...","['perfume', 'reminds', 'best', 'friend', 'actu...",2.0
3,female,Alien Mugler for women,"['white floral', 'amber', 'woody']",Imagine tripping over your own feet and fallin...,"['imagine', 'tripping', 'over', 'your', 'own',...","['imagine', 'tripping', 'over', 'your', 'own',...","['imagine', 'tripping', 'over', 'your', 'own',...","['imagine', 'tripping', 'feet', 'falling', 'fa...","['imagine', 'trip', 'foot', 'fall', 'face', 'f...",2.0
4,female,Alien Mugler for women,"['white floral', 'amber', 'woody']",Gorgeous Gorgeous Blend ..\r\nLove the scent.....,"['gorgeous', 'gorgeous', 'blend', '..', 'love'...","['gorgeous', 'gorgeous', 'blend', '', 'love', ...","['gorgeous', 'gorgeous', 'blend', 'love', 'the...","['gorgeous', 'gorgeous', 'blend', 'love', 'sce...","['gorgeous', 'gorgeous', 'blend', 'love', 'sce...",2.0


본 데이터는 모두 string 형태라 전처리 필요

In [5]:
sent_text = df['lemmatizated']

In [8]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [9]:
import re
from nltk.tokenize import word_tokenize, sent_tokenize

normalized_text = []
for string in sent_text:
     tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
     normalized_text.append(tokens)
# 각 문장에 대해서 구두점을 제거하고, 대문자를 소문자로 변환.

result = []
result = [word_tokenize(sentence) for sentence in normalized_text]

In [10]:
for line in result[:3]:
  print(line)

['get', 'sample', 'today', 'year', 'old', 'daughter', 'think', 'smell', 'spray', 'card', 'rotten', 'fish', 'nearly', 'throw', 'immediately', 'however', 'later', 'even', 'decide', 'give', 'fair', 'trial', 'spray', 'crook', 'elbow', 'rot', 'fish', 'smell', 'time', 'get', 'definite', 'bit', 'funk', 'almost', 'urine', 'scent', 'minute', 'blossom', 'gorgeous', 'smooth', 'woody', 'ambery', 'clean', 'warm', 'jasmine', 'daughter', 'didnt', 'even', 'believe', 'tell', 'fragrance', 'smell', 'earlier', 'lol', 'immediately', 'comment', 'fresh', 'clean', 'vibe', 'agree', 'soapy', 'hint', 'powdery', 'way', 'obsess', 'cant', 'stop', 'smelling', 'arm', 'glad', 'give', 'try', 'wait', 'buy', 'full', 'bottle']
['first', 'impression', 'test', 'blotter', 'heard', 'smell', 'like', 'bag', 'spray', 'like', 'gabriella', 'sabatini', 'perfume', 'everybody', 'past', 'woman', 'wear', 'smell', 'addictive', 'plasticky', 'beautiful', 'amaze', 'projection', 'longevity', 'woman', 'strong', 'personality', 'wear', 'may', 

### Word2Vec 훈련시키기

하이퍼 파라미터
* size = 워드 벡터의 특징 값. 즉, 임베딩 된 벡터의 차원.
* window = 컨텍스트 윈도우 크기
* min_count = 단어 최소 빈도 수 제한 (빈도가 적은 단어들은 학습하지 않는다.)
* workers = 학습을 위한 프로세스 수
* sg = 0은 CBOW, 1은 Skip-gram.

In [11]:
from gensim.models import Word2Vec, KeyedVectors
from gensim.models.callbacks import CallbackAny2Vec
class callback(CallbackAny2Vec): 
  """Callback to print loss after each epoch."""
  def __init__(self):
    self.epoch = 0
    self.loss_to_be_subed = 0
  def on_epoch_end(self, model):
    loss = model.get_latest_training_loss()
    loss_now = loss - self.loss_to_be_subed
    self.loss_to_be_subed = loss
    print('Loss after epoch {}: {}'.format(self.epoch, loss_now))
    self.epoch += 1


model = Word2Vec(sentences=result, size=100, window=10, min_count=2, workers=4, sg=1, compute_loss=True, iter=5, callbacks=[callback()])

Loss after epoch 0: 14573010.0
Loss after epoch 1: 11709242.0
Loss after epoch 2: 9708804.0
Loss after epoch 3: 6350464.0
Loss after epoch 4: 6250304.0


### 5 Epoch 수행 후 결과

In [12]:
model_result = model.wv.most_similar("fruity")
print(model_result)

[('citrusy', 0.7856956124305725), ('appley', 0.7781218886375427), ('juicy', 0.777457594871521), ('sweettart', 0.7770156264305115), ('birchy', 0.7749859094619751), ('pineappley', 0.7669572830200195), ('sweet', 0.7626122832298279), ('lemonpineapple', 0.7601000070571899), ('sweetsour', 0.7585804462432861), ('nisean', 0.7566289901733398)]


In [13]:
model_result = model.wv.most_similar("aquatic")
print(model_result)

[('oceanic', 0.7985668182373047), ('marine', 0.7930394411087036), ('fresh', 0.7685933113098145), ('citrus', 0.7570326924324036), ('citrusy', 0.7455393671989441), ('summery', 0.7445187568664551), ('melony', 0.7361597418785095), ('watery', 0.7260950803756714), ('freshies', 0.7217527627944946), ('ozonic', 0.7188512086868286)]


In [14]:
model_result = model.wv.most_similar("tobacco")
print(model_result)

[('vanille', 0.8290796875953674), ('pipe', 0.7958155870437622), ('honey', 0.7265146970748901), ('chergui', 0.7223398089408875), ('cinnamon', 0.7000067234039307), ('smoke', 0.684996485710144), ('spice', 0.6818305253982544), ('boclet', 0.6789635419845581), ('vanilla', 0.6701460480690002), ('booze', 0.6691057085990906)]


In [15]:
model_result = model.wv.most_similar("floral")
print(model_result)

[('florals', 0.8216738700866699), ('peachlike', 0.7838753461837769), ('lotv', 0.7654476165771484), ('flowery', 0.7654078006744385), ('overlay', 0.7636022567749023), ('citrusgreen', 0.7622569799423218), ('rosepeony', 0.7618322372436523), ('whiteairy', 0.7612645030021667), ('shampooey', 0.760265052318573), ('woodyness', 0.7591249942779541)]


In [16]:
model_result = model.wv.most_similar("orange")
print(model_result)

[('blossom', 0.8549913167953491), ('mandarin', 0.7902723550796509), ('oranges', 0.7773942947387695), ('peel', 0.7708313465118408), ('grapefruit', 0.7638823390007019), ('zest', 0.7583459615707397), ('mandarine', 0.7549320459365845), ('osmanthus', 0.7393244504928589), ('creamsicle', 0.7385150790214539), ('gingery', 0.7377530336380005)]


window size가 5일 때보다 결과가 괜찮아보임 (?)
명확한 기준을 모르겠다..ㅠㅠ

### 모델 저장 및 로드

In [19]:
model.wv.save_word2vec_format('/content/gdrive/MyDrive/Colab Notebooks/deeplearning_NLP/perfume/word embedding/model/w2v_10window') # 모델 저장
loaded_model = KeyedVectors.load_word2vec_format('/content/gdrive/MyDrive/Colab Notebooks/deeplearning_NLP/perfume/word embedding/model/w2v_10window') # 모델 로드

In [20]:
model_result = loaded_model.most_similar("orange")
print(model_result)

[('blossom', 0.8549913167953491), ('mandarin', 0.7902723550796509), ('oranges', 0.7773942947387695), ('peel', 0.7708313465118408), ('grapefruit', 0.7638823390007019), ('zest', 0.7583459615707397), ('mandarine', 0.7549320459365845), ('osmanthus', 0.7393244504928589), ('creamsicle', 0.7385150790214539), ('gingery', 0.7377530336380005)]


In [21]:
print(loaded_model.wv.similarity("ocean", "summer"))

0.51852655


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


In [22]:
print(loaded_model.wv.similarity("forest", "woody"))

0.4314216


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


In [26]:
print(loaded_model.wv.similarity("jasmine", "floral"))

0.71967846


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.
